# From Single Process to Mulitprocessing




In [2]:
import numba
import numpy as np
from numba import njit, prange, set_num_threads
 
set_num_threads(10) # setting number of cores. IMPORTANT!!!!

# setting number of cores.

@numba.njit(parallel=True)
def calculate_acceleration_parallel(X, Y, Z, Mass, G, num_bodies, i):
    ax = np.float64(0.0)
    ay = np.float64(0.0)
    az = np.float64(0.0)
    for j in prange(num_bodies):
        if j != i:
            r = ((X[j] - X[i]) ** 2 + (Y[j] - Y[i]) ** 2 + (Z[j] - Z[i]) ** 2) ** 0.5
            ax += -G * Mass[j] * (X[i] - X[j]) / (r ** 3)
            ay += -G * Mass[j] * (Y[i] - Y[j]) / (r ** 3)
            az += -G * Mass[j] * (Z[i] - Z[j]) / (r ** 3)

    return np.float64(ax), np.float64(ay), np.float64(az)


@numba.jit(nopython=True)
def calculate_acceleration_compiled(X, Y, Z, Mass, G, num_bodies, i):
    ax = np.float64(0.0)
    ay = np.float64(0.0)
    az = np.float64(0.0)
    for j in range(num_bodies):
        if j != i:
            r = ((X[j] - X[i]) ** 2 + (Y[j] - Y[i]) ** 2 + (Z[j] - Z[i]) ** 2) ** 0.5
            ax += -G * Mass[j] * (X[i] - X[j]) / (r ** 3)
            ay += -G * Mass[j] * (Y[i] - Y[j]) / (r ** 3)
            az += -G * Mass[j] * (Z[i] - Z[j]) / (r ** 3)

    return np.float64(ax), np.float64(ay), np.float64(az)


@numba.jit(parallel=True, nopython=True)
def calculate_acceleration_parallel_compiled(X, Y, Z, Mass, G, num_bodies, i):
    ax = np.float64(0.0)
    ay = np.float64(0.0)
    az = np.float64(0.0)
    for j in prange(num_bodies):
        if j != i:
            r = ((X[j] - X[i]) ** 2 + (Y[j] - Y[i]) ** 2 + (Z[j] - Z[i]) ** 2) ** 0.5
            ax += -G * Mass[j] * (X[i] - X[j]) / (r ** 3)
            ay += -G * Mass[j] * (Y[i] - Y[j]) / (r ** 3)
            az += -G * Mass[j] * (Z[i] - Z[j]) / (r ** 3)

    return np.float64(ax), np.float64(ay), np.float64(az)


def calculate_acceleration(X, Y, Z, Mass, G, num_bodies, i):
    ax = 0.0
    ay = 0.0
    az = 0.0
    for j in range(num_bodies):
        if j != i:
            r = ((X[j] - X[i]) ** 2 + (Y[j] - Y[i]) ** 2 + (Z[j] - Z[i]) ** 2) ** 0.5
            ax += -G * Mass[j] * (X[i] - X[j]) / (r ** 3)
            ay += -G * Mass[j] * (Y[i] - Y[j]) / (r ** 3)
            az += -G * Mass[j] * (Z[i] - Z[j]) / (r ** 3)
    return ax, ay, az


class Simulation:

    def __init__(self, X, Y, Z, Mass, Vx, Vy, Vz, dt, total_time, numba=True, parallel=False):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.Mass = Mass
        self.Vx = Vx
        self.Vy = Vy
        self.Vz = Vz
        self.G = 6.67430E-11
        self.dt = dt
        self.total_time = total_time
        self.time = 0
        self.num_bodies = len(X)
        self.X_history = []
        self.Y_history = []
        self.Z_history = []
        self.Vx_history = []
        self.Vy_history = []
        self.Vz_history = []
        self.numba = numba
        self.dark_matter = False
        self.parallel = parallel

    def calculate_acceleration(self, i):
        if self.numba == True:
            ax, ay, az = calculate_acceleration_compiled(self.X, self.Y, self.Z, self.Mass, self.G, self.num_bodies, i)
            if self.parallel == True:
                ax, ay, az = calculate_acceleration_parallel_compiled(self.X, self.Y, self.Z, self.Mass, self.G, self.num_bodies, i)
        elif self.parallel == True:
            ax, ay, az = calculate_acceleration_parallel(self.X, self.Y, self.Z, self.Mass, self.G, self.num_bodies, i)
        else:
            ax, ay, az = calculate_acceleration(self.X, self.Y, self.Z, self.Mass, self.G, self.num_bodies, i)
        return ax, ay, az


    def integrate(self, i):
        """
        Performs a leapfrog integration step for a single body.
        """
        ax, ay, az = self.calculate_acceleration(i)

        self.Vx[i] += ax * self.dt / 2
        self.Vy[i] += ay * self.dt / 2
        self.Vz[i] += az * self.dt / 2

        self.X[i] += self.Vx[i] * self.dt
        self.Y[i] += self.Vy[i] * self.dt
        self.Z[i] += self.Vz[i] * self.dt

        ax, ay, az = self.calculate_acceleration(i)
        self.Vx[i] += ax * self.dt / 2
        self.Vy[i] += ay * self.dt / 2
        self.Vz[i] += az * self.dt / 2

        self.time += self.dt

 

    def run_simulation(self):
        """
        Runs the simulation until the total time is reached.
        """
        import numpy as np
        from tqdm import tqdm
        self.t = []
        self.t = np.arange(0, self.total_time, self.dt)
        for i in tqdm(range(len(self.t)),total=len(self.t), desc='Running Simulation'):
            self.time = self.t[i]
            for i in range(self.num_bodies):
                self.integrate(i)
            # Store the current positions and velocities
            X_com, Y_com, Z_com = self.X[0], self.Y[0], self.Z[0]
            self.X_history.append(self.X[:].copy()-X_com)
            self.Y_history.append(self.Y[:].copy()-Y_com)
            self.Z_history.append(self.Z[:].copy()-Z_com)
            self.Vx_history.append(self.Vx[:].copy())
            self.Vy_history.append(self.Vy[:].copy())
            self.Vz_history.append(self.Vz[:].copy())

            

## Planet

In [3]:
from intial_conditions import Planets

# Set up the initial conditions

# Planets
system = Planets()
system.add_star(Mass=1.989e30)
system.add_planet(5.972e24,1.496e11)
system.add_planet(6.39e23,2.279e11)
system.add_planet(1.898e27,7.785e11)
system.add_planet(5.683e26,1.433e12)
system.add_planet(8.681e25,2.877e12)
system.add_planet(1.024e26,4.503e12)
system.add_planet(1.30900e22,5.906e12)

X = system.X
Y = system.Y
Z = system.Z
Mass = system.Mass
Vx = system.Vx
Vy = system.Vy
Vz = system.Vz


In [4]:
dt = 86400 # 1 day in seconds 
sim = Simulation(np.array(X), 
                 np.array(Y), 
                 np.array(Z), 
                 np.array(Mass), 
                 np.array(Vx), 
                 np.array(Vy), 
                 np.array(Vz), 
                 dt=dt, 
                 total_time=10*365*24*3600, 
                 numba=False, parallel=False)
sim.run_simulation()
X_run = sim.X_history

Running Simulation: 100%|██████████| 3650/3650 [00:01<00:00, 2430.42it/s]


In [5]:
dt = 86400 # 1 day in seconds 
sim = Simulation(np.array(X), 
                 np.array(Y), 
                 np.array(Z), 
                 np.array(Mass), 
                 np.array(Vx), 
                 np.array(Vy), 
                 np.array(Vz), 
                 dt=dt, 
                 total_time=10*365*24*3600, 
                 numba=False, parallel=True)
sim.run_simulation()
X_run_parallel = sim.X_history

Running Simulation: 100%|██████████| 3650/3650 [00:03<00:00, 1102.39it/s]


In [7]:
dt = 86400 # 1 day in seconds 
sim = Simulation(np.array(X), 
                 np.array(Y), 
                 np.array(Z), 
                 np.array(Mass), 
                 np.array(Vx), 
                 np.array(Vy), 
                 np.array(Vz), 
                 dt=dt, 
                 total_time=10*365*24*3600, 
                 numba=True, parallel=False)
sim.run_simulation()
X_run_numba = sim.X_history

Running Simulation: 100%|██████████| 3650/3650 [00:00<00:00, 8933.81it/s]


In [18]:
# check if the results are the same!! 
tol = 0.1 * 1.495979e+11 # 0.1 AU in meters
print(np.allclose(X_run, X_run_parallel, atol=tol))
print(np.allclose(X_run, X_run_numba, atol=tol))
print(np.allclose(X_run_parallel, X_run_numba, atol=tol))

True
True
True


In [19]:
dt = 86400 # 1 day in seconds 
sim = Simulation(np.array(X), 
                 np.array(Y), 
                 np.array(Z), 
                 np.array(Mass), 
                 np.array(Vx), 
                 np.array(Vy), 
                 np.array(Vz), 
                 dt=dt, 
                 total_time=100*365*24*3600, 
                 numba=True, parallel=True)
sim.run_simulation()

Running Simulation: 100%|██████████| 36500/36500 [00:25<00:00, 1442.74it/s]


## GALAXIES


In [20]:
# create a galaxy instead.
import numpy as np
from intial_conditions import Galaxy

pc2m = 3.086e16

galaxy = Galaxy(
    N=10000, # number of stars
    R=100*pc2m,
    z=100*pc2m,
    sigma_R=100*pc2m,
    sigma_z=100*pc2m,
    bluge_height=100*pc2m,
    bludge_frac=0.2
)

X, Y, Z = np.array(galaxy.X), np.array(galaxy.Y), np.array(galaxy.Z)
Vx, Vy, Vz = np.array(galaxy.Vx), np.array(galaxy.Vy), np.array(galaxy.Vz)
Mass = np.array(galaxy.Mass)

print(Mass)

[2.00000000e+36 1.78461389e+30 1.73333240e+30 ... 6.93821844e+31
 9.79627044e+31 3.02574781e+31]


In [21]:
dt = 86400 # 1 day in seconds
t_end = 1*365*24*3600
# takes a long time to run!!! approx 2 hours @ 26.17 seconds per time step!!!!
sim = Simulation(X, Y, Z, Vx,Vy,Vz, Mass, dt,t_end, numba=False, parallel=False)
#sim.run_simulation()

In [22]:
dt = 86400 # 1 day in seconds
t_end = 1*365*24*3600 # 1 year
# parallelis fast here, as we now only take ~1m 30s total.
sim = Simulation(X, Y, Z, Vx,Vy,Vz, Mass, dt,t_end, numba=False, parallel=True)
sim.run_simulation()

Running Simulation: 100%|██████████| 365/365 [01:16<00:00,  4.76it/s]


In [23]:
dt = 86400 # 1 day in seconds
t_end = 1*365*24*3600
# Numba is still faster here wow! 25s
sim = Simulation(X, Y, Z, Vx,Vy,Vz, Mass, dt,t_end, numba=True, parallel=False)
sim.run_simulation()

Running Simulation: 100%|██████████| 365/365 [00:25<00:00, 14.49it/s]


In [24]:
dt = 86400 # 1 day in seconds
t_end = 1*365*24*3600
# over head is still too much!!
sim = Simulation(X, Y, Z, Vx,Vy,Vz, Mass, dt,t_end, numba=True, parallel=True)
sim.run_simulation()

Running Simulation:  19%|█▉        | 71/365 [00:19<01:21,  3.61it/s]


KeyboardInterrupt: 